In [8]:
# to mount my drive for simplicity
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
import pandas as pd
import nltk
import string
from nltk.stem import PorterStemmer

w2v = {}
df = pd.read_csv('drive/MyDrive/dataset_lyrics.csv')

nltk.download('stopwords')
stop_words = set(nltk.corpus.stopwords.words('english'))
docs_without_punctuation = []
ps = PorterStemmer()

docs_without_stopwords = df['lyrics'].apply(lambda x: ' '.join([w for w in x.split() if w not in (stop_words)]))
for row in docs_without_stopwords:
  docs_without_punctuation.append(row.translate(str.maketrans('','', string.punctuation)))
stemmed = [[ps.stem(w) for w in row.split(" ")] for row in docs_without_punctuation]



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
from gensim.models import Word2Vec
model1 = Word2Vec(stemmed, min_count = 1,size = 10, window = 5, sg=0)
model1.wv.vocab

{'hold': <gensim.models.keyedvectors.Vocab at 0x7f2302eaf8e0>,
 'time': <gensim.models.keyedvectors.Vocab at 0x7f230cf9ffa0>,
 'feel': <gensim.models.keyedvectors.Vocab at 0x7f22f57d1040>,
 'break': <gensim.models.keyedvectors.Vocab at 0x7f22f57d1760>,
 'untru': <gensim.models.keyedvectors.Vocab at 0x7f22f57d1850>,
 'convinc': <gensim.models.keyedvectors.Vocab at 0x7f22f57d1e20>,
 'speak': <gensim.models.keyedvectors.Vocab at 0x7f22f57d18b0>,
 'voic': <gensim.models.keyedvectors.Vocab at 0x7f22f57d18e0>,
 'tear': <gensim.models.keyedvectors.Vocab at 0x7f22f57d1970>,
 'tri': <gensim.models.keyedvectors.Vocab at 0x7f22f57d19a0>,
 'hurt': <gensim.models.keyedvectors.Vocab at 0x7f22f57d1a30>,
 'forgiv': <gensim.models.keyedvectors.Vocab at 0x7f22f57d1ac0>,
 'okay': <gensim.models.keyedvectors.Vocab at 0x7f22f57d1b20>,
 'play': <gensim.models.keyedvectors.Vocab at 0x7f22f57d1b50>,
 'string': <gensim.models.keyedvectors.Vocab at 0x7f22f57d1be0>,
 'heart': <gensim.models.keyedvectors.Vocab at

In [11]:
print(model1.wv['fuck']) # so bekommt man vec zu einem word

[-2.183089    4.0090685   2.790015   -4.1253586  -4.3473387   0.16145393
 -0.7542506  -0.5399228  -1.2013682  -0.37026855]


In [12]:
import numpy as np

# avgerage pooling implementation
def avg_pool(doc):
    pool = np.array([0.0]*10)
    
    for idx, word in enumerate(doc):
      vec = model1.wv[word]
      pool += vec[0]

    pool = pool / len(doc)

        
    return pool

# cosine similarity implementation
def cosine_sim(a, b):
  dot = np.dot(a, b)
  aNorm = np.linalg.norm(a)
  bNorm = np.linalg.norm(b)

  denom = (aNorm * bNorm)

  if denom > 0.0:
    return dot / denom
  elif dot == 0.0:
    return 0.0

In [39]:
track_to_wv = dict()
track_to_wv_test = dict()
track_to_lyrics = dict()
#df['track_name']
#df['lyrics']

test = 10000
cnt = 0

# split dataset into test, train data
for index, row in df.iterrows():
  # prepare train data
  if cnt < test:
    docs_without_punctuation = []
    docs_without_stopwords = [w for w in row['lyrics'].split() if w not in (stop_words)]
    docs_without_punctuation = [w for w in docs_without_stopwords if w not in (string.punctuation)]

    stemmed = [[ps.stem(w) for w in roww.split(" ")] for roww in docs_without_punctuation]

    track_to_lyrics[str(row['artist_name']) + " " + str(row['track_name']) + " " + str(row['release_date'])] = stemmed
    track_to_wv[str(row['artist_name']) + " " + str(row['track_name']) + " " + str(row['release_date'])] = avg_pool(stemmed)

  # or prepare test data
  elif cnt >= test:
    docs_without_punctuation = []
    docs_without_stopwords = [w for w in row['lyrics'].split() if w not in (stop_words)]
    docs_without_punctuation = [w for w in docs_without_stopwords if w not in (string.punctuation)]

    stemmed = [[ps.stem(w) for w in roww.split(" ")] for roww in docs_without_punctuation]

    track_to_lyrics[str(row['artist_name']) + " " + str(row['track_name']) + " " + str(row['release_date'])] = stemmed
    track_to_wv_test[str(row['artist_name']) + " " + str(row['track_name']) + " " + str(row['release_date'])] = avg_pool(stemmed)
  
  if cnt == int(test*1.2):
    break
  cnt = cnt + 1

print(cnt)

12000


In [14]:
user_query = "hello i love you"

user_query_without_stopwords = []
user_query_without_punctuation = []
user_query_without_stopwords = [w for w in user_query.split() if w not in (stop_words)]
user_query_without_punctuation = [w for w in user_query_without_stopwords if w not in (string.punctuation)]

user_query_stemmed = [[ps.stem(w) for w in roww.split(" ")] for roww in user_query_without_punctuation]

print(user_query_stemmed)
print(avg_pool(user_query_stemmed))

[['hello'], ['love']]
[ 0.48751062  2.33597249  1.94590747 -3.27571213  4.10844612  0.24914847
  1.60219548  3.2726053  -1.10979375  1.77361226]


In [40]:
import torch
from torch import nn

# Create model
device = "cpu"

# Define model
class LyricsModel1(nn.Module):
    def __init__(self):
        super(LyricsModel1, self).__init__()
        self.linear = torch.nn.Sequential(
            torch.nn.Linear(10, 10),
            torch.nn.ReLU(),
            torch.nn.Linear(10, 1)
            
        )

    def forward(self, inputs_):
        x = self.linear(inputs_)
        return x

model = LyricsModel1().to(device)
model = model.float()

In [41]:
def train(data, model, loss_fn, optimizer):
  model.train()

  for key, item in data.items():
    # predict output using one hot encoding of word

    pred = model(torch.tensor(item).float())

    target = cosine_sim(avg_pool(user_query_stemmed), item)

    #print(pred)
    #print(torch.tensor([target]).float())

    # Compute prediction error
    loss = loss_fn(pred, torch.tensor([target]).float())

    # Backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [43]:
def test(data, model, loss_fn, optimizer):
  model.eval()

  for key, item in data.items():
    # predict output using one hot encoding of word

    pred = model(torch.tensor(item).float())

    target = cosine_sim(avg_pool(user_query_stemmed), item)

    print(pred)
    print(torch.tensor([target]).float())

    # Compute prediction error
    loss = loss_fn(pred, torch.tensor([target]).float())
    print(loss)
    print()
  return

In [44]:
optimizer = torch.optim.Adagrad(model.parameters(), lr=0.2)
loss_fn = torch.nn.BCEWithLogitsLoss()

epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(track_to_wv, model, loss_fn, optimizer)
    test(track_to_wv_test, model, loss_fn, optimizer)
print("Done!")

Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
tensor([0.7241])
tensor(0.5904, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)

tensor([0.6438], grad_fn=<AddBackward0>)
tensor([0.6347])
tensor(0.6574, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)

tensor([0.9112], grad_fn=<AddBackward0>)
tensor([0.7207])
tensor(0.5924, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)

tensor([0.6454], grad_fn=<AddBackward0>)
tensor([0.6775])
tensor(0.6298, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)

tensor([1.1305], grad_fn=<AddBackward0>)
tensor([0.7616])
tensor(0.5493, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)

tensor([0.5748], grad_fn=<AddBackward0>)
tensor([0.5827])
tensor(0.6863, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)

tensor([1.0389], grad_fn=<AddBackward0>)
tensor([0.7527])
tensor(0.5599, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)

tensor([0.6465], grad_fn=<AddBackward0>)
tensor([0.6686])
tensor(0.6355, grad_fn=<BinaryCrossEntropyWithLogitsBac

In [45]:
model = LyricsModel1().to(device)
model = model.float()

# use mseloss because this is not a classification task
# also change optimizer for testing
optimizer = torch.optim.SGD(model.parameters(), lr=0.05)
loss_fn = torch.nn.MSELoss()

epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(track_to_wv, model, loss_fn, optimizer)
    test(track_to_wv_test, model, loss_fn, optimizer)
print("Done!")

Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
tensor([0.7241])
tensor(0.0003, grad_fn=<MseLossBackward0>)

tensor([0.6276], grad_fn=<AddBackward0>)
tensor([0.6347])
tensor(5.0711e-05, grad_fn=<MseLossBackward0>)

tensor([0.7048], grad_fn=<AddBackward0>)
tensor([0.7207])
tensor(0.0003, grad_fn=<MseLossBackward0>)

tensor([0.6624], grad_fn=<AddBackward0>)
tensor([0.6775])
tensor(0.0002, grad_fn=<MseLossBackward0>)

tensor([0.7616], grad_fn=<AddBackward0>)
tensor([0.7616])
tensor(2.3194e-09, grad_fn=<MseLossBackward0>)

tensor([0.5810], grad_fn=<AddBackward0>)
tensor([0.5827])
tensor(2.9188e-06, grad_fn=<MseLossBackward0>)

tensor([0.7532], grad_fn=<AddBackward0>)
tensor([0.7527])
tensor(3.0880e-07, grad_fn=<MseLossBackward0>)

tensor([0.6660], grad_fn=<AddBackward0>)
tensor([0.6686])
tensor(6.7016e-06, grad_fn=<MseLossBackward0>)

tensor([0.5080], grad_fn=<AddBackward0>)
tensor([0.5285])
tensor(0.0004, grad_fn=<MseLossBackward0>)

tensor([0.7721], grad_fn=<AddBackwar

In [46]:
# Create model
device = "cpu"

# Define 2nd model with more layers
class LyricsModel2(nn.Module):
    def __init__(self):
        super(LyricsModel2, self).__init__()
        self.linear = torch.nn.Sequential(
            torch.nn.Linear(10, 20),
            torch.nn.ReLU(),
            torch.nn.Linear(20, 10),
            torch.nn.ReLU(),
            torch.nn.Linear(10, 1)
        )

    def forward(self, inputs_):
        x = self.linear(inputs_)
        return x

model = LyricsModel2().to(device)
model = model.float()

In [47]:
model = LyricsModel2().to(device)
model = model.float()

# use mseloss because this is not a classification task
# also change optimizer for testing
optimizer = torch.optim.SGD(model.parameters(), lr=0.05)
loss_fn = torch.nn.MSELoss()

epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(track_to_wv, model, loss_fn, optimizer)
    test(track_to_wv_test, model, loss_fn, optimizer)
print("Done!")

Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
tensor([0.7241])
tensor(9.8690e-05, grad_fn=<MseLossBackward0>)

tensor([0.6245], grad_fn=<AddBackward0>)
tensor([0.6347])
tensor(0.0001, grad_fn=<MseLossBackward0>)

tensor([0.7140], grad_fn=<AddBackward0>)
tensor([0.7207])
tensor(4.4787e-05, grad_fn=<MseLossBackward0>)

tensor([0.6801], grad_fn=<AddBackward0>)
tensor([0.6775])
tensor(6.4494e-06, grad_fn=<MseLossBackward0>)

tensor([0.7721], grad_fn=<AddBackward0>)
tensor([0.7616])
tensor(0.0001, grad_fn=<MseLossBackward0>)

tensor([0.5773], grad_fn=<AddBackward0>)
tensor([0.5827])
tensor(2.9390e-05, grad_fn=<MseLossBackward0>)

tensor([0.7537], grad_fn=<AddBackward0>)
tensor([0.7527])
tensor(1.1278e-06, grad_fn=<MseLossBackward0>)

tensor([0.6433], grad_fn=<AddBackward0>)
tensor([0.6686])
tensor(0.0006, grad_fn=<MseLossBackward0>)

tensor([0.4893], grad_fn=<AddBackward0>)
tensor([0.5285])
tensor(0.0015, grad_fn=<MseLossBackward0>)

tensor([0.7697], grad_fn=<AddBackwar